
## Principes

### Machine Learning ou Econométrie ?

Un modèle statistique est une construction mathématique qui formalise une loi
ayant généré les données. La différence principale entre *machine learning* (ML) et économétrie
est dans le degré de structure imposé par le modélisateur :

- En ML,
la structure imposée par le *data scientist* est minimale et ce sont plutôt
les algorithmes qui, sur des critères de performance statistique, vont
déterminer une loi mathématique qui correspond aux données.

- En économétrie,
les hypothèses de structure des lois sont plus fortes (même dans un cadre semi ou non-paramétrique) et sont plus souvent imposées
par le modélisateur.

L'adoption du Machine Learning dans la littérature économique a été longue car la structuration des données est souvent le pendant empirique d'hypothèses théoriques sur le comportement des acteurs ou des marchés (Athey and Imbens, 2019).
Ces hypothèses correspondent bien au concept de modèle statistique, coeur de l'économétrie, 
mais moins bien à l'approche algorithmique du *machine learning* 
où la loi du processus générateur des données est secondaire. 

Dans une conception caricaturale, et fausse, l’économétrie s’attacherait
à comprendre la causalité de certaines variables sur une autre 
quand le *machine learning* se contenterait d'essayer de bons modèles prédictifs
en exploitant les relations de corrélations entre les variables. 
Cette vision caricaturale est partielle car il existe des manières 
de comprendre l'influence de facteurs sur une variable d'intérêt en machine learning
(en étudiant les *variables importance* par exemple) tout comme on peut voir 
l'économétrie comme un enjeu de prédiction, notamment lorsqu'on s'intéresse
à des phénomènes catégoriels. 
La réelle différence est plutôt dans la démarche de recherche. 
Les économètres tendent à s'intéresser beaucoup principalement à la question de 
la distribution des effets et sa potentielle hétérogénéité 
alors que les praticiens du Machine Learning qui vont
plutôt s'intéresser à la distribution des erreurs de prédiction.

### Apprentissage supervisé ou non supervisé ?

On distingue généralement deux types de méthodes, selon qu'on dispose d'information, dans l'échantillon
d'apprentissage, sur les valeurs cibles *y* (on utilisera parfois le terme *label*) :

* **apprentissage supervisé** : la valeur cible est connue et peut-être utilisée pour évaluer la qualité d'un modèle 

*Ex : modèles de prédiction du type régression / classification : SVM, kNN, arbres de classification...*

* **apprentissage non supervisé** : la valeur cible est inconnue et ce sont des critères statistiques qui vont amener
à sélectionner la structure de données la plus plausible. 

*Ex : modèles de réduction de dimension ou de clustering (PCA, kmeans...)*

## Panorama d'un éco-système vaste

Grâce aux principaux packages de Machine Learning (`scikit`), Deep Learning (`keras`, `pytorch`, `TensorFlow`...) et économétrie  (`statsmodels`), la modélisation est extrêmement simplifiée. Cela ne doit pas faire oublier l'importance de la structuration et de la préparation des données. Souvent, l'étape la plus cruciale est le choix du modèle le plus adapté à la structure des données.

L'aide-mémoire suivante, issue de l'aide de `scikit-learn`, concernant les modèles de Machine Learning peut déjà donner de premiers enseignements sur les différentes familles de modèles:

![](https://scikit-learn.org/stable/_static/ml_map.png)
## Données

### Quelques exemples de *Machine Learning* avec *open-data*

Trouver un jeu de données public de qualité pour effectuer un travail
d'apprentissage peut parfois être pénible. Il est souvent nécessaire
de combiner plusieurs sources de données pour disposer d'un jeu de données
approprié.

`Etalab` a mis en ligne un [catalogue](https://datascience.etalab.studio/dgml/)
des jeux de données présents sur `data.gouv` en fonction de critères de 
choix de modélisation. Il peut être utile de s'y référer de temps en temps. 

### Données utilisées en fil rouge de cette partie

La plupart des exemples de cette partie s'appuient sur les résultats des
élections US 2020 au niveau comtés. Plusieurs bases sont utilisées pour 
cela:

* Les données électorales sont une reconstruction à partir des données du *MIT Election Lab*
proposées sur `Github` par [@tonmcg](https://github.com/tonmcg/US_County_Level_Election_Results_08-20)
(sur [zenodo](https://zenodo.org/record/3975765))
ou directement disponibles sur le site du [MIT Election Lab](https://electionlab.mit.edu/data)
* Les données socioéconomiques (population, données de revenu et de pauvreté, 
taux de chômage, variables d'éducation) proviennent de l'USDA ([source](https://www.ers.usda.gov/data-products/county-level-data-sets/))
* Le *shapefile* vient des données du *Census Bureau*. Le fichier peut
être téléchargé directement depuis cet url:
<https://www2.census.gov/geo/tiger/GENZ2019/shp/cb_2019_us_county_20m.zip>

Le code pour construire une base unique à partir de ces sources diverses
est disponible ci-dessous : 

```{.python}
import urllib
import urllib.request
import os
import zipfile
from urllib.request import Request, urlopen
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd

def download_url(url, save_path):
    with urllib.request.urlopen(url) as dl_file:
        with open(save_path, 'wb') as out_file:
            out_file.write(dl_file.read())


def create_votes_dataframes():
    
  Path("data").mkdir(parents=True, exist_ok=True)
  
  
  download_url("https://www2.census.gov/geo/tiger/GENZ2019/shp/cb_2019_us_county_20m.zip", "data/shapefile")
  with zipfile.ZipFile("data/shapefile", 'r') as zip_ref:
      zip_ref.extractall("data/counties")
  
  shp = gpd.read_file("data/counties/cb_2019_us_county_20m.shp")
  shp = shp[~shp["STATEFP"].isin(["02", "69", "66", "78", "60", "72", "15"])]
  shp
  
  df_election = pd.read_csv("https://raw.githubusercontent.com/tonmcg/US_County_Level_Election_Results_08-20/master/2020_US_County_Level_Presidential_Results.csv")
  df_election.head(2)
  population = pd.read_excel("https://www.ers.usda.gov/webdocs/DataFiles/48747/PopulationEstimates.xls?v=290.4", header = 2).rename(columns = {"FIPStxt": "FIPS"})
  education = pd.read_excel("https://www.ers.usda.gov/webdocs/DataFiles/48747/Education.xls?v=290.4", header = 4).rename(columns = {"FIPS Code": "FIPS", "Area name": "Area_Name"})
  unemployment = pd.read_excel("https://www.ers.usda.gov/webdocs/DataFiles/48747/Unemployment.xls?v=290.4", header = 4).rename(columns = {"fips_txt": "FIPS", "area_name": "Area_Name", "Stabr": "State"})
  income = pd.read_excel("https://www.ers.usda.gov/webdocs/DataFiles/48747/PovertyEstimates.xls?v=290.4", header = 4).rename(columns = {"FIPStxt": "FIPS", "Stabr": "State", "Area_name": "Area_Name"})
  
  
  dfs = [df.set_index(['FIPS', 'State']) for df in [population, education, unemployment, income]]
  data_county = pd.concat(dfs, axis=1)
  df_election = df_election.merge(data_county.reset_index(), left_on = "county_fips", right_on = "FIPS")
  df_election['county_fips'] = df_election['county_fips'].astype(str).str.lstrip('0')
  shp['FIPS'] = shp['GEOID'].astype(str).str.lstrip('0')
  votes = shp.merge(df_election, left_on = "FIPS", right_on = "county_fips")
  
  req = Request('https://dataverse.harvard.edu/api/access/datafile/3641280?gbrecs=false')
  req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
  content = urlopen(req)
  df_historical = pd.read_csv(content, sep = "\t")
  #df_historical = pd.read_csv('https://dataverse.harvard.edu/api/access/datafile/3641280?gbrecs=false', sep = "\t")
  
  df_historical = df_historical.dropna(subset = ["FIPS"])
  df_historical["FIPS"] = df_historical["FIPS"].astype(int)
  df_historical['share'] = df_historical['candidatevotes']/df_historical['totalvotes']
  df_historical = df_historical[["year", "FIPS", "party", "candidatevotes", "share"]]
  df_historical['party'] = df_historical['party'].fillna("other")
  
  df_historical_wide = df_historical.pivot_table(index = "FIPS", values=['candidatevotes',"share"], columns = ["year","party"])
  df_historical_wide.columns = ["_".join(map(str, s)) for s in df_historical_wide.columns.values]
  df_historical_wide = df_historical_wide.reset_index()
  df_historical_wide['FIPS'] = df_historical_wide['FIPS'].astype(str).str.lstrip('0')
  votes['FIPS'] = votes['GEOID'].astype(str).str.lstrip('0')
  votes = votes.merge(df_historical_wide, on = "FIPS")
  votes["winner"] =  np.where(votes['votes_gop'] > votes['votes_dem'], 'republican', 'democrats') 

  return votes
```

Je l'ai également mis à disposition sur
<a href="https://raw.githubusercontent.com/linogaliana/python-datascientist/master/content/course/modelisation/get_data.py" class="github"><i class="fab fa-github"></i></a>

## Contenu de la partie

{{< list_children >}}

Des éléments viendront ultérieurement enrichir cette partie, notamment
dans le domaine économétrique : 

* maximum vraisemblance
* stats bayésiennes
* semi et non paramétrique: méthodes noyaux, GAM

## Références

Athey, S., & Imbens, G. W. (2019). Machine learning methods economists should know about, arxiv.